<a href="https://colab.research.google.com/github/VictorRamires/datascience/blob/main/k_means_credito.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [59]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import plotly.express as px
import numpy as np
from sklearn.decomposition import PCA
base_cartao = pd.read_csv('/content/credit_card_clients.csv', header = 1)
base_cartao

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,1,20000,2,2,1,24,2,2,-1,-1,...,0,0,0,0,689,0,0,0,0,1
1,2,120000,2,2,2,26,-1,2,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,3,90000,2,2,2,34,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,4,50000,2,2,1,37,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,5,50000,1,2,1,57,-1,0,-1,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,29996,220000,1,3,1,39,0,0,0,0,...,88004,31237,15980,8500,20000,5003,3047,5000,1000,0
29996,29997,150000,1,3,2,43,-1,-1,-1,-1,...,8979,5190,0,1837,3526,8998,129,0,0,0
29997,29998,30000,1,2,2,37,4,3,2,-1,...,20878,20582,19357,0,0,22000,4200,2000,3100,1
29998,29999,80000,1,3,1,41,1,-1,0,0,...,52774,11855,48944,85900,3409,1178,1926,52964,1804,1


In [60]:
#base_cartao['BILL_TOTAL'] = base_cartao['BILL_AMT1'] + base_cartao['BILL_AMT2'] + base_cartao['BILL_AMT3'] + base_cartao['BILL_AMT4'] + base_cartao['BILL_AMT5'] + base_cartao['BILL_AMT6']
base_cartao['BILL_TOTAL'] = base_cartao.loc[:, ['BILL_AMT1','BILL_AMT2', 'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6']].sum(axis=1)
base_cartao

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month,BILL_TOTAL
0,1,20000,2,2,1,24,2,2,-1,-1,...,0,0,0,689,0,0,0,0,1,7704
1,2,120000,2,2,2,26,-1,2,0,0,...,3455,3261,0,1000,1000,1000,0,2000,1,17077
2,3,90000,2,2,2,34,0,0,0,0,...,14948,15549,1518,1500,1000,1000,1000,5000,0,101653
3,4,50000,2,2,1,37,0,0,0,0,...,28959,29547,2000,2019,1200,1100,1069,1000,0,231334
4,5,50000,1,2,1,57,-1,0,-1,0,...,19146,19131,2000,36681,10000,9000,689,679,0,109339
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,29996,220000,1,3,1,39,0,0,0,0,...,31237,15980,8500,20000,5003,3047,5000,1000,0,725349
29996,29997,150000,1,3,2,43,-1,-1,-1,-1,...,5190,0,1837,3526,8998,129,0,0,0,21182
29997,29998,30000,1,2,2,37,4,3,2,-1,...,20582,19357,0,0,22000,4200,2000,3100,1,70496
29998,29999,80000,1,3,1,41,1,-1,0,0,...,11855,48944,85900,3409,1178,1926,52964,1804,1,266611


In [61]:
#variavel com o limite do cartão e a divida total do cartão de credito

x_cartao = base_cartao.iloc[:,[1,25]].values
x_cartao

array([[ 20000,   7704],
       [120000,  17077],
       [ 90000, 101653],
       ...,
       [ 30000,  70496],
       [ 80000, 266611],
       [ 50000, 230874]])

In [62]:
#deixando os dados do x_cartão na mesma escala, e depois deixando na mesma escala do scaler_cartao
scaler_cartao = StandardScaler()
x_cartao = scaler_cartao.fit_transform(x_cartao)
x_cartao

array([[-1.13672015, -0.69069198],
       [-0.3659805 , -0.66599747],
       [-0.59720239, -0.44316987],
       ...,
       [-1.05964618, -0.52525745],
       [-0.67427636, -0.00856436],
       [-0.90549825, -0.10271861]])

In [63]:
#usando o metodo wcss para descobrir o numero de clusters
#o WCSS é o somatorio do quadrado das distancias entre cada ponto e o centroide
#nesse caso, testamos de 1 a 10 centroides
#escolhemos o numero de centroides a partir do momento em que não há uma queda
#significativa no gráfico
wcss = []
for i in range(1,11):
  kmeans_cartao = KMeans(n_clusters=i, random_state = 0)
  kmeans_cartao.fit(x_cartao)
  wcss.append(kmeans_cartao.inertia_)

wcss

[59999.99999999978,
 35196.57080583435,
 20128.132558347686,
 14943.809173865899,
 10707.537016753624,
 8603.395384848965,
 7400.069599689482,
 6353.473303353463,
 5665.823857740179,
 5052.188166408734]

In [64]:
#grafico de linha para visualizar o wcss
grafico = px.line(x = range(1,11), y = wcss)
grafico.show()

In [65]:
#conforme visualizamos no grafico, podemos fazer o algoritmo com 4 ou 5 clusters
kmeans_cartao = KMeans(n_clusters = 4, random_state = 0) #random_state = 0 para ter sempre os mesmos resultados
rotulos = kmeans_cartao.fit_predict(x_cartao)
grafico = px.scatter(x = x_cartao[:,0], y = x_cartao[:,1], color = rotulos)
grafico.show()

Portanto, em X temos o limite, quanto mais pra direita, maior o limite, já em Y temos o tamanho da divida, quanto mais pra cima, maior o gasto. O grupo azul por exemplo, é o grupo de pessoas que tem o limite do cartão de crédito baixo
o grupo rosa por exemplo, são pessoas que tem o um limite alto, mas não gastam muito. O grupo laranja é composto por pessoas que gastam muito e tem limite alto. Podemos perceber um outlier com limite muito alto que gasta muito no topo direito do grafico.

In [66]:
lista_clientes = np.column_stack((base_cartao, rotulos))
lista_clientes
#gerando uma lista com todos os clientes concatenando a listas de base_cartao e rotulos

array([[     1,  20000,      2, ...,      1,   7704,      0],
       [     2, 120000,      2, ...,      1,  17077,      0],
       [     3,  90000,      2, ...,      0, 101653,      0],
       ...,
       [ 29998,  30000,      1, ...,      1,  70496,      0],
       [ 29999,  80000,      1, ...,      1, 266611,      0],
       [ 30000,  50000,      1, ...,      1, 230874,      0]])

In [67]:
#ordenando a lista de clientes pelo numero do rótulo
lista_clientes = lista_clientes[lista_clientes[:,26].argsort()]
lista_clientes

array([[     1,  20000,      2, ...,      1,   7704,      0],
       [ 16789,  30000,      1, ...,      1,  14274,      0],
       [ 16788,  30000,      1, ...,      1,  90453,      0],
       ...,
       [ 20882, 110000,      1, ...,      0, 554308,      3],
       [ 20744, 220000,      1, ...,      0, 452961,      3],
       [ 20270, 150000,      2, ...,      0, 546939,      3]])

In [68]:
base_cartao.columns

Index(['ID', 'LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_0',
       'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2',
       'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1',
       'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6',
       'default payment next month', 'BILL_TOTAL'],
      dtype='object')

In [69]:
#vamos começar uma analisa agora utilizando mais colunas da nossa tabela
x_cartao_mais = base_cartao.iloc[:, [1,2,3,4,5,25]]
x_cartao_mais

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,BILL_TOTAL
0,20000,2,2,1,24,7704
1,120000,2,2,2,26,17077
2,90000,2,2,2,34,101653
3,50000,2,2,1,37,231334
4,50000,1,2,1,57,109339
...,...,...,...,...,...,...
29995,220000,1,3,1,39,725349
29996,150000,1,3,2,43,21182
29997,30000,1,2,2,37,70496
29998,80000,1,3,1,41,266611


In [70]:
#padronização dos dados com calculo da média e desvio padrão
scaler_cartao_mais = StandardScaler
x_cartao_mais = scaler_cartao.fit_transform(x_cartao_mais)
x_cartao_mais

array([[-1.13672015,  0.81016074,  0.18582826, -1.05729503, -1.24601985,
        -0.69069198],
       [-0.3659805 ,  0.81016074,  0.18582826,  0.85855728, -1.02904717,
        -0.66599747],
       [-0.59720239,  0.81016074,  0.18582826,  0.85855728, -0.16115646,
        -0.44316987],
       ...,
       [-1.05964618, -1.23432296,  0.18582826,  0.85855728,  0.16430256,
        -0.52525745],
       [-0.67427636, -1.23432296,  1.45111372, -1.05729503,  0.59824792,
        -0.00856436],
       [-0.90549825, -1.23432296,  0.18582826, -1.05729503,  1.14067961,
        -0.10271861]])

In [71]:
#calculo do wcss 
wcss_mais = []
for i in range(1,11):
  kmeans_cartao = KMeans(n_clusters = i, random_state = 0)
  kmeans_cartao.fit(x_cartao_mais)
  wcss_mais.append(kmeans_cartao.inertia_)

wcss_mais

[180000.00000000038,
 143649.08688195905,
 123753.68521018763,
 108637.12912356813,
 95861.3242235647,
 86985.2745778842,
 80306.53189000252,
 76132.86228447755,
 72237.32509871296,
 69020.66723762438]

In [72]:
#grafico de linha para visualizar o wcss
grafico_wcss_mais = px.line(x = range(1,11), y = wcss_mais)
grafico_wcss_mais.show()

In [73]:
#vamos tentar inicialmente com 4 clusters
kmeans_cartao_mais = KMeans(n_clusters = 4, random_state = 0)
rotulos = kmeans_cartao_mais.fit_predict(x_cartao_mais)
rotulos

array([1, 0, 0, ..., 3, 1, 1], dtype=int32)

Como não conseguimos criar um gráfico de tamanho 2 já que temos 6 atributos,
iremos utilizar uma tecnica cahamada PCA - Principal Component Analysis, que é utlizada para redução de dimensionalidade. Serão realizados calculos que me geram só dois atributos como resultado.

In [76]:
pca = PCA(n_components=2)
x_cartao_mais_pca = pca.fit_transform(x_cartao_mais)
x_cartao_mais_pca.shape #gerao os atributos atraves da combinação dos 6 já existentes

(30000, 2)

In [77]:
x_cartao_mais_pca

array([[-0.74082054, -1.13671858],
       [-1.48027121, -0.30100547],
       [-0.94737386, -0.48666789],
       ...,
       [-0.79468657, -0.90012663],
       [ 1.17562376, -1.54746987],
       [ 1.13614987, -1.14039836]])

In [80]:
grafico_pca = px.scatter(x = x_cartao_mais_pca[:,0], y = x_cartao_mais_pca[:,1], color = rotulos)
grafico_pca.show()

In [82]:
lista_cliente_mais = np.column_stack((base_cartao, rotulos))
lista_cliente_mais = lista_cliente_mais[lista_cliente_mais[:,26].argsort()]
lista_cliente_mais

array([[ 22235, 250000,      2, ...,      0, 120233,      0],
       [  6902, 300000,      2, ...,      0, 605733,      0],
       [  6903, 180000,      2, ...,      0,  33111,      0],
       ...,
       [  7847,  30000,      1, ...,      1, 168918,      3],
       [  7866,  50000,      1, ...,      0, 132306,      3],
       [ 15000,  50000,      1, ...,      0, 249711,      3]])